<a href="https://colab.research.google.com/github/Manish-k723/100-days-of-ML/blob/main/Kaggle_challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opendatasets --quiet

In [2]:
import opendatasets as od
od.download('https://www.kaggle.com/competitions/playground-series-s3e16/data')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: manishkumar7432698
Your Kaggle Key: ··········


100%|██████████| 2.65M/2.65M [00:00<00:00, 148MB/s]


Extracting archive ./playground-series-s3e16/playground-series-s3e16.zip to ./playground-series-s3e16


In [3]:
import pandas as pd
import numpy as np

In [4]:
train_df = pd.read_csv('/content/playground-series-s3e16/train.csv')
test_df = pd.read_csv('/content/playground-series-s3e16/test.csv')
sub_df = pd.read_csv('/content/playground-series-s3e16/sample_submission.csv')

In [7]:
train_df.drop('id', axis = 1, inplace = True)
test_df.drop('id', axis = 1, inplace = True)

In [14]:
train_df = pd.get_dummies(train_df,columns = ['Sex'])

In [15]:
test_df = pd.get_dummies(test_df,columns = ['Sex'])

In [19]:
X = train_df.drop('Age',axis = 1)
y = train_df['Age']

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.15, random_state = 42)

In [37]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [39]:
!pip install catboost --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.4 MB/s eta 0:00:00


In [41]:
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor

In [24]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [25]:
from sklearn.metrics import r2_score, mean_squared_error

In [48]:
models = {
    'lr' : LinearRegression(),
    'ridge':Ridge(),
    'lasso':Lasso(),
    'lightgbm':LGBMRegressor(),
    'cat': CatBoostRegressor(verbose = False)
}

In [49]:
scores = []

In [50]:
for i in range(len(list(models.keys()))):
  model = list(models.values())[i]
  model.fit(X_train,y_train)
  train_pred = model.predict(X_train)
  val_pred = model.predict(X_val)

  r2_train = r2_score(y_train, train_pred)
  r2_val = r2_score(y_val, val_pred)

  rmse_train = np.sqrt(mean_squared_error(y_train,train_pred))
  rmse_val = np.sqrt(mean_squared_error(y_val, val_pred))

  scores.append([[list(models.keys())[i]],r2_val, rmse_val])

  print(r2_train, rmse_train)
  print(r2_val, rmse_val)
  print('\n')

0.5496713840845244 2.127962053019237
0.5570802172496268 2.1287140733801837


0.5496703737532498 2.127964440104281
0.5570539172894174 2.1287772724814125


0.4009714757615028 2.4542729349552026
0.4010583159789807 2.4754095491413484


0.6200896190655778 1.9545180568265605
0.5939611673924564 2.038161424031033


0.6440966287317674 1.8917561634648368
0.5941467263934712 2.037695652796442




In [128]:
model = CatBoostRegressor(verbose = False)
model.fit(X,y)
np.sqrt(mean_squared_error(model.predict(X_val), y_val))

1.9006037817000323

In [129]:
pred = model.predict(X_val).astype(int)

In [130]:
r2_score(pred, y_val)

0.3604989216438498

In [131]:
np.sqrt(mean_squared_error(pred, y_val))

1.9898011474541357

In [132]:
test_pred = model.predict(test_df)

In [133]:
sub_df['Age'] = test_pred

In [134]:
sub_df

,id,Age
0,74051,7.742535
1,74052,7.736341
2,74053,10.858948
3,74054,9.504857
4,74055,7.515128
...,...,...
49363,123414,9.532351
49364,123415,8.080605
49365,123416,13.355883
49366,123417,9.861401


In [135]:
sub_df.to_csv('submission2.csv', index = False)

In [67]:
X_train.shape, y_train.shape

((62943, 10), (62943,))

In [69]:
lr = LinearRegression()

In [70]:
lr.fit(X_train, y_train)

LinearRegression()

In [71]:
lr.coef_

array([ 0.91884941,  2.16202633,  7.03209139,  0.20068428, -0.6195616 ,
       -0.216322  ,  0.50043773,  0.37300447, -0.65178826,  0.27878379])

In [72]:
lr.intercept_

3.382269734783236

In [73]:
y_pred = lr.predict(X_val)
r2_score(y_pred, y_val)

0.19268566832757483

In [108]:
class GDRegressor:
  
  def __init__(self, learning_rate = 0.01, epochs = 100):
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.coef_ = None
    self.intercept_ = None

  def fit(self, X_train, y_train):
    self.intercept_ = 0
    self.coef_ = np.ones(X_train.shape[1])
    
    for i in range(self.epochs):
      y_hat = np.dot(X_train, self.coef_) + self.intercept_
      intercept_der = -2*np.mean(y_train - y_hat)
      self.intercept_ = self.intercept_ - self.learning_rate * intercept_der

      coef_der = -2*np.dot((y_train - y_hat),X_train)/X_train.shape[0]
      self.coef_ = self.coef_ - self.learning_rate * coef_der
    # print(self.coef_, self.intercept_)

  def predict(self, X_test):
    return np.dot(X_test, self.coef_) + self.intercept_

In [102]:
obj = GDRegressor(learning_rate = 0.001, epochs = 800)

In [103]:
obj.fit(X_train, y_train)

[ 1.64344138  1.48475401  1.16448037 -0.02865135 -0.42759077  0.36551688
  0.9871552   1.14035687  1.47579538  1.18246523] 0.7986174711807067


In [104]:
y_pred = obj.predict(X_val)

In [ ]:
for i in range(1, 12):
    epoch = i*100
    lr = [0.001,0.0001]
    for rate in lr:
      gd = GDRegressor(learning_rate = rate, epochs = epoch)
      gd.fit(X_train, y_train)
      pred = gd.predict(X_val)
      print(f'Iteration: {i} with lr is {rate} and r2 is {r2_score(pred, y_val)}')
      print('='*30)

In [111]:
gdc = GDRegressor(learning_rate = 0.001, epochs = 500)

In [112]:
gdc.fit(X_train, y_train)

In [113]:
pred = gdc.predict(X_val)

In [114]:
r2_score(pred, y_val)

0.3455982901588167

In [116]:
test_pred = gdc.predict(test_df)

In [118]:
sub_df['Age'] = test_pred

In [120]:
sub_df.to_csv('MyGD.csv', index = False)

In [121]:
gdc.fit(X, y)

In [122]:
test_pred = gdc.predict(test_df)

In [123]:
sub_df['Age'] = test_pred

In [124]:
sub_df.to_csv('MyGD1.csv', index = False)

In [125]:
record = []

In [ ]:
for i in range(33, 45):
    epoch = i*100
    gd = GDRegressor(learning_rate = 0.0001, epochs = epoch)
    gd.fit(X_train, y_train)
    pred = gd.predict(X_val)
    score = r2_score(pred, y_val)
    print(f'Iteration: {i} with lr is {0.0001} and r2 is {score}')
    print('='*30)
    record.append(score)

In [136]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
param_grid = {
    'iterations': [200, 500, 800, 1000,1200],
    'learning_rate': [0.01, 0.1, 0.3],
    'border_count': [50, 100, 150,200],
    'depth': [4, 6, 8]
}

catboost = CatBoostRegressor(verbose = False)

randomize_search = RandomizedSearchCV(estimator=catboost, param_distributions=param_grid, cv=5)

randomize_search.fit(X_train, y_train)

best_params = randomize_search.best_params_
best_score = randomize_search.best_score_

print(best_params, best_score)
best_catboost = CatBoostRegressor(**best_params)

best_catboost.fit(X_train, y_train)

In [138]:
best_params

{'learning_rate': 0.1, 'iterations': 1000, 'depth': 4, 'border_count': 200}

In [ ]:
best_catboost = CatBoostRegressor(**best_params)
best_catboost.fit(X,y)

In [142]:
test_pred = best_catboost.predict(test_df)

In [144]:
sub_df['Age'] = test_pred

In [146]:
sub_df.to_csv('best_cat.csv', index = False)